# A Gentle Introduction to RAG Applications

This notebook creates a simple RAG (Retrieval-Augmented Generation) system to answer questions from a PDF document using an open-source model.

In [11]:
PDF_FILE = "Honda_2020.pdf"

# We'll be using Llama 3.1 8B for this example.
MODEL = "llama3.2:latest"

## PDF to markdown

Picking all documents from the pdf folder and transcribing them

<img src='images/documents.png' width="1000">

In [ ]:
import pymupdf4llm
import os

local_path = os.getcwd()
PDF_folder = "PDF_montadoras/Amostra de conteúdo - PDF_s montadoras"
Transcript_folder = "PDF_montadoras/markdown_transcription"


for folder in os.listdir(os.path.join(local_path, PDF_folder)):
    llama_docs = pymupdf4llm.to_markdown(os.path.join(local_path, PDF_folder, folder))
    print(folder)
    with open(os.path.join(local_path, Transcript_folder, f"{folder}.md"), "w") as md_file:
        md_file.write(llama_docs)


Processing /home/joao/Projetos/pessoais/local_rag/PDF_montadoras/Amostra de conteúdo - PDF_s montadoras/Fiat_2016.pdf...
[                                        ] (0/22[                                        ] (  1/22[                                        ] (  2/22[                                        ] (  3/22[                                        ] (  4/22[                                        ] (  5/228[=                                       ] (  6/2[=                                       ] (  7/2[=                                       ] (  8/2[=                                       ] (  9/2[=                                       ] ( 10/228[=                                       ] ( 11/22[==                                      ] ( 12/22[==                                      ] ( 13/22[==                                      ] ( 14/22[==                                      ] ( 15/22[==                                      ] ( 16/22[==                              

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_FILE)
pages = loader.load()

print(f"Number of pages: {len(pages)}")
print(f"Length of a page: {len(pages[1].page_content)}")
print("Content of a page:", pages[1].page_content)

ImportError: pypdf package not found, please install it with `pip install pypdf`

## Splitting the pages in chunks

Pages are too long, so let's split pages into different chunks.

<img src='images/splitter.png' width="1000">


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)

chunks = splitter.split_documents(pages)
print(f"Number of chunks: {len(chunks)}")
print(f"Length of a chunk: {len(chunks[1].page_content)}")
print("Content of a chunk:", chunks[1].page_content)


Number of chunks: 23
Length of a chunk: 1358
Content of a chunk: took better advantage of it than Stripe. At YC we use the term
"Collison installation" for the technique they invented. More
diﬀident founders ask "Will you try our beta?" and if the answer
is yes, they say "Great, we'll send you a link." But the Collison
brothers weren't going to wait. When anyone agreed to try Stripe
they'd say "Right then, give me your laptop" and set them up on
the spot.
There are two reasons founders resist going out and recruiting
users individually. One is a combination of shyness and laziness.
They'd rather sit at home writing code than go out and talk to a
bunch of strangers and probably be rejected by most of them.
But for a startup to succeed, at least one founder (usually the
CEO) will have to spend a lot of time on sales and marketing. [2]
The other reason founders ignore this path is that the absolute
numbers seem so small at ﬁrst. This can't be how the big, famous
startups got started, they

## Storing the chunks in a vector store

We can now generate embeddings for every chunk and store them in a vector store.

<img src='images/vectorstore.png' width="1000">


In [ ]:
print(chunks)

[Document(metadata={'source': 'Paul.pdf', 'page': 0}, page_content="Want to start a startup? Get funded by Y Combinator.\nJuly 2013\nOne of the most common types of advice we give at Y Combinator\nis to do things that don't scale. A lot of would-be founders believe\nthat startups either take oﬀ or don't. You build something, make\nit available, and if you've made a better mousetrap, people beat a\npath to your door as promised. Or they don't, in which case the\nmarket must not exist. [1]\nActually startups take oﬀ because the founders make them take\noﬀ. There may be a handful that just grew by themselves, but\nusually it takes some sort of push to get them going. A good\nmetaphor would be the cranks that car engines had before they\ngot electric starters. Once the engine was going, it would keep\ngoing, but there was a separate and laborious process to get it\ngoing.\nRecruit\nThe most common unscalable thing founders have to do at the\nstart is to recruit users manually. Nearly all s

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model=MODEL)
vectorstore = FAISS.from_documents(chunks, embeddings)

/tmp/ipykernel_33285/2008811563.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=MODEL)


## Setting up a retriever

We can use a retriever to find chunks in the vector store that are similar to a supplied question.

<img src='images/retriever.png' width="1000">



In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("What can you get away with when you only have a small number of users?")

[Document(metadata={'source': 'paul.pdf', 'page': 7}, page_content="started are not merely a necessary evil, but change the companypermanently for the better. If you have to be aggressive aboutuser acquisition when you're small, you'll probably still beaggressive when you're big. If you have to manufacture your ownhardware, or use your software on users's behalf, you'll learnthings you couldn't have learned otherwise. And mostimportantly, if you have to work hard to delight users when youonly have a handful of them, you'll keep doing it when you have alot."),
 Document(metadata={'source': 'paul.pdf', 'page': 0}, page_content='Want to start a startup? Get funded by Y Combinator.'),
 Document(metadata={'source': 'paul.pdf', 'page': 7}, page_content='Notes[1] Actually Emerson never mentioned mousetraps specifically. Hewrote "If a man has good corn or wood, or boards, or pigs, tosell, or can make better chairs or knives, crucibles or churchorgans, than anybody else, you will find a broad h

## Configuring the model

We'll be using Ollama to load the local model in memory. After creating the model, we can invoke it with a question to get the response back.

<img src='images/model.png' width="1000">

In [ ]:
from langchain_ollama import ChatOllama

model = ChatOllama(model=MODEL, temperature=0)
model.invoke("Who is the president of the United States?")

AIMessage(content='As of my last update in April 2023, Joe Biden is the President of the United States. He took office on January 20, 2021, succeeding Donald Trump as the 46th President of the United States. Please note that this information might change over time due to elections or other political developments.', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-11T15:56:24.740654949Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1628813229, 'load_duration': 22064348, 'prompt_eval_count': 20, 'prompt_eval_duration': 40750000, 'eval_count': 65, 'eval_duration': 1520747000}, id='run-c453f60f-b4c3-4ed8-84db-c715f94eb563-0', usage_metadata={'input_tokens': 20, 'output_tokens': 65, 'total_tokens': 85})

## Parsing the model's response

The response from the model is an `AIMessage` instance containing the answer. We can extract the text answer by using the appropriate output parser. We can connect the model and the parser using a chain.

<img src='images/parser.png' width="1000">


In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
print(chain.invoke("Who is the president of the United States?"))

As of my last update in April 2023, Joe Biden is the President of the United States. He took office on January 20, 2021, succeeding Donald Trump as the 46th President of the United States. Please note that this information might change over time due to elections or other political developments.


## Setting up a prompt

In addition to the question we want to ask, we also want to provide the model with the context from the PDF file. We can use a prompt template to define and reuse the prompt we'll use with the model.


<img src='images/prompt.png' width="1000">

In [ ]:
from langchain.prompts import PromptTemplate

template = """
You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: Here is some context

Question: Here is a question



## Adding the prompt to the chain

We can now chain the prompt with the model and the parser.

<img src='images/chain1.png' width="1000">

In [ ]:
chain = prompt | model | parser

chain.invoke({
    "context": "Anna's sister is Susan", 
    "question": "Who is Susan's sister?"
})


'Anna.'

## Adding the retriever to the chain

Finally, we can connect the retriever to the chain to get the context from the vector store.

<img src='images/chain2.png' width="1000">

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

## Using the chain to answer questions

Finally, we can use the chain to ask questions that will be answered using the PDF document.

In [ ]:
questions = [
    "What can you get away with when you only have a small number of users?",
    "What's the most common unscalable thing founders have to do at the start?",
    "What's one of the biggest things inexperienced founders and investors get wrong about startups?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print("*************************\n")

Question: What can you get away with when you only have a small number of users?
Answer: Delight users.
*************************

Question: What's the most common unscalable thing founders have to do at the start?
Answer: Delighting users individually.
*************************

Question: What's one of the biggest things inexperienced founders and investors get wrong about startups?
Answer: They want to seem big too early and imitate flaws of big companies, like indifference to individual users.
*************************

